In [1]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

In [2]:
attacks_original = pd.read_csv('../data/attacks.csv', encoding = 'latin1')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

attacks = attacks_original.copy()

In [3]:
attacks_original.shape

(25723, 24)

In [4]:
attacks.columns = attacks.columns.str.strip().str.lower().str.replace(' ','_').str.replace('.','_').str.replace(':','')

In [5]:
nan_cols = attacks.isna().mean() * 100

round(nan_cols[nan_cols > 0], 2)

case_number                66.17
date                       75.50
year                       75.51
type                       75.52
country                    75.69
area                       77.27
location                   77.60
activity                   77.62
name                       76.32
sex                        77.70
age                        86.51
injury                     75.61
fatal_(y/n)                77.60
time                       88.54
species                    86.53
investigator_or_source     75.57
pdf                        75.50
href_formula               75.50
href                       75.50
case_number_1              75.50
case_number_2              75.50
original_order             75.47
unnamed_22                100.00
unnamed_23                 99.99
dtype: float64

In [6]:
attacks = attacks.dropna(thresh = 3) #de esta manera nos quedamos con las filas que al menos tienen 3 columnas con datos.
attacks.shape

(6302, 24)

In [7]:
nan_cols = attacks.isna().mean() * 100

round(nan_cols[nan_cols > 0], 2)

case_number                0.02
year                       0.03
type                       0.06
country                    0.79
area                       7.22
location                   8.57
activity                   8.63
name                       3.33
sex                        8.97
age                       44.92
injury                     0.44
fatal_(y/n)                8.55
time                      53.22
species                   45.03
investigator_or_source     0.27
href_formula               0.02
unnamed_22                99.98
unnamed_23                99.97
dtype: float64

In [8]:
attacks.fillna({'type':'unknown', 'location' :  'unknown','name':'unknown', 'sex':'unknown','species':'unknown', 'injury':'not registered', 'fatal_(y/n)':'N/A',
                'investigator_or_source':'unknown', 'pdf':'not applies','href_formula': 'not applies',
                'href': 'not applies', 'unnamed_22': 0, 'unnamed_23': 0}, inplace = True)

In [9]:
nan_cols = attacks.isna().mean() * 100

round(nan_cols[nan_cols > 0], 2)

case_number     0.02
year            0.03
country         0.79
area            7.22
activity        8.63
age            44.92
time           53.22
dtype: float64

In [10]:
attacks.age = round(pd.to_numeric(attacks.age, errors='coerce'),0) #para convertir todo a float y en caso de no ser float, devuelveme un NAN

In [11]:
attacks.age.fillna(attacks.age.mean(), inplace = True) #para rellenar todos los valores nulos por la media de la columna

In [12]:
attacks.age = attacks.age.astype('int')

In [13]:
nan_cols = attacks.isna().mean() * 100

round(nan_cols[nan_cols > 0], 2)

case_number     0.02
year            0.03
country         0.79
area            7.22
activity        8.63
time           53.22
dtype: float64

In [14]:
activitymode = attacks.activity.mode()

In [15]:
attacks.activity.fillna('activitymode', inplace=True)

In [16]:
nan_cols = attacks.isna().mean() * 100

round(nan_cols[nan_cols > 0], 2)

case_number     0.02
year            0.03
country         0.79
area            7.22
time           53.22
dtype: float64

In [17]:
attacks.case_number=[int(e) for e in range(1, len(attacks)+1)]

In [18]:
nan_cols = attacks.isna().mean() * 100

round(nan_cols[nan_cols > 0], 2)

year        0.03
country     0.79
area        7.22
time       53.22
dtype: float64

In [19]:
attacks.date = attacks.date.str.replace('Before ', '')

In [20]:
attacks.date[attacks.date.apply(len)!=11] = np.nan

In [21]:
attacks.dropna(subset=['date'], inplace=True)

In [22]:
nan_cols = attacks.isna().mean() * 100

round(nan_cols[nan_cols > 0], 2)

country     0.31
area        4.19
time       41.92
dtype: float64

In [23]:
attacks.year = attacks.year.astype('int')

In [24]:
attacks.time = attacks.time.str.replace('h',':')

In [25]:
attacks.time.replace(to_replace = r'[a-zA-Z-/&\(\)."">? ]', value = '', regex = True, inplace = True)

In [26]:
attacks.time.fillna('0', inplace = True)

In [27]:
attacks.time[attacks.time.apply(len)!=5] = np.nan

In [28]:
timemode = attacks.time.mode()
timemode

0    11:00
Name: time, dtype: object

In [29]:
attacks.time.fillna('11:00', inplace = True)

In [30]:
nan_cols = attacks.isna().mean() * 100

round(nan_cols[nan_cols > 0], 2)

country    0.31
area       4.19
dtype: float64

In [31]:
attacks.loc[(attacks.country.isnull()) & (attacks.area.notnull())]

,case_number,date,year,type,country,area,location,activity,name,sex,age,injury,fatal_(y/n),time,species,investigator_or_source,pdf,href_formula,href,case_number_1,case_number_2,original_order,unnamed_22,unnamed_23
3387,3388,02-Aug-1970,1970,Invalid,NaN,Caribbean Sea,Between St. Kitts & Nevis,Sea Disaster Sinking of ferryboat Christina,unknown,unknown,27,"Sharks scavenged on bodies, but no record of t...",N/A,11:00,Shark involvement prior to death was not confi...,"Rome News Tribune, 8/3/1970",1970.08.02-Christina-ferryboat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1970.08.02,1970.08.02,2916.0,0,0
4018,4019,26-Jan-1960,1960,Sea Disaster,NaN,"Between Timor & Darwin, Australia",unknown,Portuguese Airliner with 9 people aboard went ...,unknown,unknown,27,"As searchers approached wreckage, sharks circl...",N,11:00,unknown,"V.M. Coppleson (1962), p.260",1960.01.26-Portuguese airliner.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1960.01.26,1960.01.26,2285.0,0,0
4231,4232,13-Sep-1956,1956,Unprovoked,NaN,Near the Andaman & Nicobar Islands,unknown,Climbing back on ship,male,M,27,FATAL,Y,11:00,Blue shark,M. Hosina,1956.09.13-TunaBoat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1956.09.13,1956.09.13,2072.0,0,0
5742,5743,30-Jul-1887,1887,Sea Disaster,NaN,Gulf of California,unknown,"Sea disaster, wreck of the sloop Sara",2 males,M,27,FATAL,Y,11:00,unknown,"Austin Weekly Statesman, 8/11/1887",1887.07.30-TheSara.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1887.07.30,1887.07.30,561.0,0,0
6137,6138,05-Jul-1787,1787,Unprovoked,NaN,St Helena,Landing Place,Swimming,Private Isaac Hicksled,M,27,FATAL,Y,11:00,unknown,"H.R. Janisch (1885), Extracts from the St. Hel...",1787.07.05-Hicksled.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1787.07.05,1787.07.05,166.0,0,0


In [32]:
attacks.at[3387,'country'] = 'ST KITTS AND NEVIS'
attacks.at[4018,'country'] = 'AUSTRALIA'
attacks.at[4231,'country'] = 'INDIA'
attacks.at[5742,'country'] = 'USA'
attacks.at[6137,'country'] = 'ENGLAND'

In [33]:
attacks.loc[(attacks.country.isnull()) & (attacks.location!='unknown')]

,case_number,date,year,type,country,area,location,activity,name,sex,age,injury,fatal_(y/n),time,species,investigator_or_source,pdf,href_formula,href,case_number_1,case_number_2,original_order,unnamed_22,unnamed_23
3605,3606,21-Oct-1965,1965,Unprovoked,NaN,NaN,Florida Strait,The boat Caribou II sank,Mario Castellanos,M,39,Survived,N,11:00,unknown,"Lodi News Sentinel, 10/30/1965",1965.10.21-Castellanos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1965.10.21,1965.10.21,2698.0,0,0
4639,4640,24-Oct-1944,1944,Sea Disaster,NaN,NaN,225 miles east of Hong Kong,Japanese POW ship Arisan Maru with 1800 Americ...,unknown,M,27,Most of the men drowned & some were taken by s...,Y,17:30,unknown,internet (multiple),1944.10.24-ArisanMaru.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1944.10.24,1944.10.24,1664.0,0,0
5425,5426,17-Jan-1909,1909,Invalid,NaN,NaN,Near the equator,Jumped overboard,Thomas Butler,M,36,FATAL,N/A,11:00,Questionable,"Star, 3/18/1909",1909.01.17-Butler.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1909.01.17,1909.01.17,878.0,0,0
5810,5811,25-Jul-1881,1881,Unprovoked,NaN,NaN,Santa Cruz,Bathing,Father Hudson,M,27,Survived,N,11:00,unknown,"Grey River Argus,10/3/1881, p.2",1881.06.25-FatherHudson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1881.06.25,1881.06.25,493.0,0,0
6155,6156,17-Dec-1742,1742,Unprovoked,NaN,NaN,Carlisle Bay,Swimming,2 impressed seamen,M,27,FATAL,Y,11:00,unknown,"C. Moore, GSAF",1742.12.17-AdviceSeamen.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1742.12.17,1742.12.17,148.0,0,0


In [34]:
attacks.at[3605,'country'] = 'USA'
attacks.at[3605,'area'] = 'Florida'

attacks.at[4639,'country'] = 'JAPAN'
attacks.at[4639,'area'] = 'Hong Kong'

attacks.at[5425,'country'] = 'unknown'
attacks.at[5425,'area'] = 'unknown'

attacks.at[5810,'country'] = 'USA'
attacks.at[5810,'area'] = 'California'

attacks.at[6155,'country'] = 'BARBADOS'
attacks.at[6155,'area'] = 'Antigua y Barbuda'

In [35]:
nan_cols = attacks.isna().mean() * 100

round(nan_cols[nan_cols > 0], 2)

country    0.10
area       4.09
dtype: float64

In [36]:
attacks.country.fillna('unknown', inplace = True)
attacks.area.fillna('unknown', inplace = True)

In [37]:
attacks.head()

,case_number,date,year,type,country,area,location,activity,name,sex,age,injury,fatal_(y/n),time,species,investigator_or_source,pdf,href_formula,href,case_number_1,case_number_2,original_order,unnamed_22,unnamed_23
0,1,25-Jun-2018,2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18:00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,0,0
1,2,18-Jun-2018,2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,11:00,unknown,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,0,0
2,3,09-Jun-2018,2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07:45,unknown,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,0,0
3,4,08-Jun-2018,2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,27,Minor injury to lower leg,N,11:00,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,0,0
4,5,04-Jun-2018,2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,27,Lacerations to leg & hand shark PROVOKED INCIDENT,N,11:00,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,0,0


In [38]:
attacks.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4769 entries, 0 to 6290
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   case_number             4769 non-null   int64  
 1   date                    4769 non-null   object 
 2   year                    4769 non-null   int32  
 3   type                    4769 non-null   object 
 4   country                 4769 non-null   object 
 5   area                    4769 non-null   object 
 6   location                4769 non-null   object 
 7   activity                4769 non-null   object 
 8   name                    4769 non-null   object 
 9   sex                     4769 non-null   object 
 10  age                     4769 non-null   int32  
 11  injury                  4769 non-null   object 
 12  fatal_(y/n)             4769 non-null   object 
 13  time                    4769 non-null   object 
 14  species                 4769 non-null   

In [39]:
for c in attacks.select_dtypes('integer'):
    
    attacks[c]=pd.to_numeric(attacks[c], downcast='integer')

In [40]:
for c in attacks.select_dtypes('float'):
    
    attacks[c]=pd.to_numeric(attacks[c], downcast='float')

In [41]:
for c in attacks.select_dtypes('object'):
    
    attacks[c]=attacks[c].astype('category')
    
attacks.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4769 entries, 0 to 6290
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   case_number             4769 non-null   int16   
 1   date                    4769 non-null   category
 2   year                    4769 non-null   int16   
 3   type                    4769 non-null   category
 4   country                 4769 non-null   category
 5   area                    4769 non-null   category
 6   location                4769 non-null   category
 7   activity                4769 non-null   category
 8   name                    4769 non-null   category
 9   sex                     4769 non-null   category
 10  age                     4769 non-null   int8    
 11  injury                  4769 non-null   category
 12  fatal_(y/n)             4769 non-null   category
 13  time                    4769 non-null   category
 14  species                 